In [48]:
print('hello supervised learning')

hello supervised learning


In [49]:
import polars as pl

## Load 

Load from CSV and clean up the data
1. Remove all rows where the year is 2014 because its the first year there are no pbs
2. Clean up gender as it contains "M" or "Male" and "F" or "Female". Just take the first character for simplity
3. Data Types

In [50]:
results_with_pb_df = pl.read_csv("data/withpb_and_age-all-results.csv")


results_with_pb_df = results_with_pb_df.drop("XC_Count_of_event", "XC_Position_Min", "XC_Position_Max", "XC_Min_Age", "XC_Max_Age","exists", "from_file")

# Gender Clean up
results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("Gender").str.slice(0, 1).alias("Gender")
)

results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("is_pb").cast(pl.Boolean),
    pl.col("first_time_run").cast(pl.Boolean)
)

display(results_with_pb_df)




Position,Time,time_in_seconds,Bib Number,Name,Gender,Gender Position,Club,is_club_member,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_run_attempts,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_time_in_seconds,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,prior_quickest_vs_run_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_year,position_record_worst_year,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob
i64,str,f64,i64,str,str,f64,str,bool,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64
3,"""00:29:14""",1754.0,880,"""Andrew Masters""","""M""",3.0,"""Lewes AC""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:39.250000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,24.0,16.0,8.0,12.0,12.0,12.0,10.0,10.0,10.0,1.0,1.0,1.0,0.0,0.0,2014.0,2014.0,12.0,12.0,12.0,"""Andrew""","""Masters""","""V50""","""V50""","""V50""","""M""","""Lewes/West 4/Wimbledon Windmil…",80881.0,"""match""",1.0,"""all""",40.0,44.0,42.0,1974.0,1970.0,1972.0
4,"""00:29:34""",1774.0,588,"""Paul Cousins""","""M""",4.0,"""Haywards Heath Harriers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:41.750000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,9.0,5.0,4.0,2.0,5.0,7.0,10.0,10.0,10.0,3.0,3.0,1.0,0.0,0.0,2014.0,2014.0,2.0,5.0,7.0,"""Paul""","""Cousins""","""V60""","""V60""","""V60""","""M""","""Haywards Heath""",9998.0,"""match""",1.0,"""all""",50.0,54.0,52.0,1964.0,1960.0,1962.0
5,"""00:29:47""",1787.0,902,"""Michael Neville""","""M""",5.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:43.375000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Michael""","""Neville""","""V45""","""V45""","""V45""","""M""","""Chichester""",43629.0,"""match""",1.0,"""all""",35.0,39.0,37.0,1979.0,1975.0,1977.0
6,"""00:30:07""",1807.0,918,"""Conrad Meagher""","""M""",6.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:45.875000""",2.0,2.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Conrad""","""Meagher""","""SEN""","""SEN""","""SEN""","""M""","""Havant""",566746.0,"""match""",1.0,"""no_club""",13.0,24.0,18.5,2001.0,1990.0,1995.5
7,"""00:30:09""",1809.0,530,"""John Russell""","""M""",7.0,"""Portslade Hedgehoppers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:46.125000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,23.0,15.0,8.0,10.0,27.0,72.0,10.0,9.0,8.0,10.0,10.0,1.0,0.0,0.0,2014.0,2014.0,10.0,27.0,72.0,null,null,"""TBC""","""TBC""","""TBC""","""n""",null,null,"""missing""",0.0,"""not found""",null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",2861.0,631,"""Ann Savidge""","""F""",303.0,"""Burgess 

## Do we have balanced target classes?

We calculate the PB from the prior runs. 
- The first year is 2014 which has no pbs because there is no prior data
- This needs to be removed as its unbalances the data

We also need to remove any first time only runs as a first is always a pb (technically) and will unbalance the data

In [51]:
results_with_pb_df_all = results_with_pb_df

# Pivot the DataFrame
pivot_df = results_with_pb_df_all.pivot(
    values=["is_pb", "first_time_run"],
    index="year",
    on="is_pb",
    aggregate_function="len"
).sort("year")


display(pivot_df)

# Remove all rows where the year is 2014
results_with_pb_df = results_with_pb_df.filter(
    (pl.col("year") != 2014) & (pl.col("first_time_run") == False)
)

year,is_pb_false,is_pb_true,first_time_run_false,first_time_run_true
i64,u32,u32,u32,u32
2014,6811,null,6811,null
2015,6289,1116,6289,1116
2016,5925,1283,5925,1283
2017,5263,1042,5263,1042
2018,5269,904,5269,904
2019,5451,1172,5451,1172
2020,378,93,378,93
2022,3967,637,3967,637
2023,3758,631,3758,631


## Find Categoricals

These are columns with limited subset of values.

For example **education** could be 
- Graduate
- Not Graduate

In our data set
- Club 
- RaceName
- Gender
All of which are unlikely to influence a pb

However, I'll add of these are categorical variables to the model.
- RaceName
- Gender

The two `to_dummies` colum will turn these into columns with 1s and 0s.
- So for `RaceName` it will create a column for each unique value in the column and set it to 1 if the value is present and 0 if not.
- `RaceName_Downland Dash`
- `RaceName_Fittleworth`
- for each race combo





In [52]:
results_with_pb_with_dummies_df = results_with_pb_df.to_dummies(
    ["Gender", "RaceName", 'p_of_10_road_age_category','is_club_member'], drop_first=True
)


In [53]:
display(results_with_pb_with_dummies_df)

Position,Time,time_in_seconds,Bib Number,Name,Gender_F,Gender Position,Club,is_club_member_false,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,date,distance,pace,club_position,…,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category_SEN,p_of_10_road_age_category_TBC,p_of_10_road_age_category_U11,p_of_10_road_age_category_U13,p_of_10_road_age_category_U15,p_of_10_road_age_category_U17,p_of_10_road_age_category_U20,p_of_10_road_age_category_U23,p_of_10_road_age_category_V35,p_of_10_road_age_category_V40,p_of_10_road_age_category_V45,p_of_10_road_age_category_V50,p_of_10_road_age_category_V55,p_of_10_road_age_category_V65,p_of_10_road_age_category_V70,p_of_10_road_age_category_V75,p_of_10_road_age_category_V80,p_of_10_road_age_category_V85,p_of_10_road_age_category_V95,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob
i64,str,f64,i64,str,u8,f64,str,u8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,str,f64,str,f64,…,f64,f64,str,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64
7,"""00:29:11""",1751.0,167,"""Paul Cousins""",0,7.0,"""Haywards Heath Harriers""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:38.875000""",1.0,…,19.0,132.0,"""Paul""","""Cousins""","""V60""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""V60""","""M""","""Haywards Heath""",9998.0,"""match""",1.0,"""all""",51.0,55.0,53.0,1964.0,1960.0,1962.0
13,"""00:29:54""",1794.0,203,"""Ben Stock""",0,13.0,"""Saints and Sinners""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:44.250000""",2.0,…,19.0,35.0,"""Ben""","""Stock""","""V35""",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"""V35""","""M""","""Ealing Southall & Middx""",4844.0,"""many""",5.0,"""no_club""",26.0,30.0,28.0,1989.0,1985.0,1987.0
19,"""00:30:24""",1824.0,294,"""Adrian Scott""",0,19.0,"""Steyning AC""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:48""",1.0,…,56.0,160.0,null,null,"""TBC""",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""TBC""","""n""",null,null,"""missing""",0.0,"""not found""",null,null,null,null,null,null
21,"""00:30:29""",1829.0,390,"""Gary Witton""",0,21.0,"""Worthing Harriers""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:48.625000""",4.0,…,17.0,21.0,"""Gary""","""Witton""","""V50""",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"""V50""","""M""","""Worthing""",59314.0,"""match""",1.0,"""all""",41.0,45.0,43.0,1974.0,1970.0,1972.0
24,"""00:30:45""",1845.0,295,"""Scott Chinchen""",0,24.0,"""Steyning AC""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:50.625000""",2.0,…,41.0,94.0,"""Scott""","""Chinchen""","""V35""",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"""V35""","""M""","""Steyning""",462082.0,"""match""",1.0,"""all""",26.0,30.0,28.0,1989.0,1985.0,1987.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
615,"""00:45:24""",2724.0,203,"""Verity Dods""",1,291.0,"""Horsham Joggers""",0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",6.54,"""0:06:56.513761""",38.0,…,429.0,686.0,"""Verity""","""Dods""","""V50""",0,0,0,0,0,0,0,0,0,

## Split the data

Remove the target column from the features. **is_pb**

Features are the columns that will be used to predict the target variable. 

In this case, the features are all the columns that **Known before the race starts**
- Prior performance 
- Gender
- Club mates (you would see them before the start and on course that could inspire you to run faster)
 


In [83]:
pb_achieved_features_df = results_with_pb_with_dummies_df.drop("is_pb")

# Info about the RACE or runner that is not useful for prediction  
#   runner
pb_achieved_features_df = pb_achieved_features_df.drop("Bib Number", "Name", "Club")
pb_achieved_features_df = pb_achieved_features_df.drop(  'p_of_10_firstname', 'p_of_10_surname', 'p_of_10_track_age_category', 'p_of_10_xc_age_category', 'p_of_10_gender', 'p_of_10_club', 'p_of_10_athlete_id')
#   race
pb_achieved_features_df = pb_achieved_features_df.drop("date", 'year', 'month', 'day', 'distance' )

# achieved time position which would be unknown at the start of the race! 
pb_achieved_features_df = pb_achieved_features_df.drop( "Time", "pace", 'Position', 'time_in_seconds','Gender Position','Points', 'club_position', 'club_position_gender', 'first_time_run' )

# Prior info on years which are versus the current year or just contains the year
pb_achieved_features_df = pb_achieved_features_df.drop( "prior_quickest_run_in_year", "prior_time_vs_current_change", "prior_time_vs_current_time", "prior_quickest_run_time", 'prior_quickest_vs_run_time', 'position_record_best_year', 'position_record_worst_year' )

pb_achieved_features_df = pb_achieved_features_df.drop(  'match_type',  'matches','parameters_used', 'min_age', 'max_age', 'average_age', 'min_dob', 'max_dob', 'avg_dob')

print(pb_achieved_features_df.columns)

display(pb_achieved_features_df)

pb_achieved_target_df = results_with_pb_df.select("is_pb")


['Gender_F', 'is_club_member_false', 'RaceName_Downland Dash', 'RaceName_Fittleworth', 'RaceName_Great Walstead 5', 'RaceName_Gunpowder Trot', 'RaceName_Hangover-5', 'RaceName_Hedgehoppers 5', 'RaceName_Hickstead Gallop', 'RaceName_Highdown Hike', 'RaceName_Hornets Stinger', 'RaceName_Hove Park', 'RaceName_Lewes AC Easter Monday', 'RaceName_Run Your Heart Out 8k', 'RaceName_Seven Stiles', 'RaceName_Steepdown Challenge', 'RaceName_Steyning AC Roundhill Romp', 'RaceName_Tilgate 5', 'RaceName_Trundle View', 'RaceName_Valentine', 'RaceName_Windlesham-Whip', 'RaceName_Worthing Lido', 'prior_run_attempts', 'prior_quickest_run_time_in_seconds', 'prior_quickest_run_position', 'fellow_club_mates_running', 'fellow_club_mates_running_male', 'fellow_club_mates_running_female', 'best_position_year_to_date', 'average_position_year_to_date', 'worst_position_year_to_date', 'best_points_year_to_date', 'average_points_year_to_date', 'worst_points_year_to_date', 'runs_in_wsfrl_year_to_date', 'wsfrl_tenur

Gender_F,is_club_member_false,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,prior_run_attempts,prior_quickest_run_time_in_seconds,prior_quickest_run_position,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_road_age_category_SEN,p_of_10_road_age_category_TBC,p_of_10_road_age_category_U11,p_of_10_road_age_category_U13,p_of_10_road_age_category_U15,p_of_10_road_age_category_U17,p_of_10_road_age_category_U20,p_of_10_road_age_category_U23,p_of_10_road_age_category_V35,p_of_10_road_age_category_V40,p_of_10_road_age_category_V45,p_of_10_road_age_category_V50,p_of_10_road_age_category_V55,p_of_10_road_age_category_V65,p_of_10_road_age_category_V70,p_of_10_road_age_category_V75,p_of_10_road_age_category_V80,p_of_10_road_age_category_V85,p_of_10_road_age_category_V95
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1774.0""","""4.0""",13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,1.0,0.0,2.0,19.0,132.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1870.0""","""13.0""",46.0,25.0,21.0,8.0,17.0,27.0,10.0,9.0,9.0,9.0,11.0,2.0,1.0,1.0,8.0,19.0,35.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1851.0""","""10.0""",15.0,10.0,5.0,10.0,59.0,160.0,10.0,8.0,5.0,7.0,8.0,2.0,1.0,1.0,10.0,56.0,160.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1890.0""","""16.0""",29.0,18.0,11.0,12.0,16.0,19.0,10.0,10.0,10.0,5.0,6.0,1.0,1.0,1.0,12.0,17.0,21.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1911.0""","""22.0""",15.0,10.0,5.0,22.0,38.0,94.0,10.0,8.0,7.0,9.0,11.0,2.0,1.0,1.0,22.0,41.0,94.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0,"""2666.0""","""686.0""",37.0,14.0,23.0,285.0,469.0,686.0,1.0,1.0,1.0,7.0,19.0,2.0,2.0,1.0,226.0,429.0,686.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,"""3039.0""","""534.0""",42.0,26.0,16.0,289.0,499.0,635.0,1.0,1.0,1.0,3.0,6.0,2.0,2.0,1.0,278.0,450.0,635.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,"""2499.0""","""657.0""",72.0,35.0,37.0,440.0,544.0,657.0,2.0,1.0,1.0,3.0,5.0,2.0,2.0,1.0,305.0,465.0,657.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 123

(
    pb_achieved_feature_train_df,
    pb_achieved_feature_test_df,
    pb_achieved_target_train_df,
    pb_achieved_target_test_df,
) = train_test_split(
    pb_achieved_features_df,
    pb_achieved_target_df,
    test_size=0.30,
    random_state=SEED,
)



## Scale the data

It is important to scale the data so that the model is not distorted by really large values.

### My notes are 
- Dimensionality reduction, which 
1. Rotates the data. Some like 45 degrees
2. Keeps the same data but reduce the complexity
- Principal Component Analysis (PCA) is a technique that can be used to reduce the dimensionality of the data.

### Notes from Copilot
The StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

In [86]:
scaler = StandardScaler()

pb_achieved_feature_train_scaled_df = scaler.fit_transform(
    pb_achieved_feature_train_df
)

pb_achieved_feature_test_scaled_df = scaler.fit_transform(
    pb_achieved_feature_test_df
)

Do we have balanced target classes?

# Logistic regression example

```
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```

## Constraints

> ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values


One common strategy is to use the mean or median of the column to fill in the missing values. Here is how you can do it using SimpleImputer from sklearn.impute:  
Import SimpleImputer.
Create an imputer instance with the desired strategy (e.g., mean, median).
Fit the imputer on the training data and transform both the training and test data.

```
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Create an imputer instance with the mean strategy
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the training data and transform both the training and test data
pb_achieved_feature_train_scaled_df = imputer.fit_transform(pb_achieved_feature_train_scaled_df)
pb_achieved_feature_test_scaled_df = imputer.transform(pb_achieved_feature_test_scaled_df)

# Create and fit the logistic regression classifier
logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```





In [88]:
# Convert Polars DataFrame to Pandas DataFrame
pb_achieved_feature_train_df_pandas = pb_achieved_feature_train_df.to_pandas()
pb_achieved_target_train_df_pandas = pb_achieved_target_train_df.to_pandas()
pb_achieved_feature_test_df_pandas = pb_achieved_feature_test_df.to_pandas()
pb_achieved_target_test_df_pandas = pb_achieved_target_test_df.to_pandas()



# Define Column for Defaults

**All columns** need **default values** for **missing values**.

This can have quite an impact on the model. For example if 
- You defaulted missing **points to 10** which is the **highest point** you can score 
- You would **create an artificially high amount** of high points scores when the data is much lower

The second section of the code checks the column exist in the data frame. If they do not exist you will get an error.

> ValueError: A given column is not a column of the dataframe 

In [89]:
# Define the columns
# "Position", "Gender Position", 
time_in_seconds_columns = [ "prior_quickest_run_time_in_seconds"]

years_since_value_columns = ["position_record_years_since_best", "position_record_years_since_worst"]

low_value_good_columns = ["best_position_year_to_date", "average_position_year_to_date", "worst_position_year_to_date", "position_record_best_position_all_time", "position_record_worst_position_all_time", "prior_quickest_run_position",  "position_record_average_position_all_time"]

high_value_good_columns = ["fellow_club_mates_running", "fellow_club_mates_running_male", "fellow_club_mates_running_female", "average_points_year_to_date", "best_points_year_to_date", "worst_points_year_to_date", "runs_in_wsfrl_year_to_date", 'wsfrl_tenure_in_runs','wsfrl_tenure_in_years', 'prior_run_attempts']


binary_columns = [col for col in pb_achieved_feature_train_df_pandas.columns 
                  if col.startswith('RaceName_') or col.startswith('p_of_10_road_age_category_')]
binary_columns.append("is_club_member_false")
binary_columns.append("Gender_F")



# Combine the lists
all_feature_columns = set(low_value_good_columns 
                          + high_value_good_columns 
                          + binary_columns 
                          
                          + time_in_seconds_columns 
                          + years_since_value_columns)



# Find columns that are not in the combined list
extra_columns = [col for col in pb_achieved_feature_train_df_pandas.columns if col not in all_feature_columns]
print(f"non range columns in low_value_good_columns: {extra_columns}")

# Check if columns exist in the DataFrame
missing_columns = [col for col in all_feature_columns if col not in pb_achieved_feature_train_df_pandas.columns]
missing_low_value_good_columns = [col for col in low_value_good_columns if col not in pb_achieved_feature_train_df_pandas.columns]
missing_high_value_good_columns = [col for col in high_value_good_columns if col not in pb_achieved_feature_train_df_pandas.columns]


# Print the results
if missing_columns:
    print(f"Missing columns in WHOLE set: {missing_columns}")
else:
    print("All feature columns exist in the DataFrame.")

if missing_low_value_good_columns:
    print(f"Missing columns in low_value_good_columns: {missing_low_value_good_columns}")
else:
    print("All columns in low_value_good_columns exist in the DataFrame.")

if missing_high_value_good_columns:
    print(f"Missing columns in high_value_good_columns: {missing_high_value_good_columns}")
else:
    print("All columns in high_value_good_columns exist in the DataFrame.")
    
    
pb_achieved_feature_train_df_pandas.head(4)

non range columns in low_value_good_columns: []
All feature columns exist in the DataFrame.
All columns in low_value_good_columns exist in the DataFrame.
All columns in high_value_good_columns exist in the DataFrame.


,Gender_F,is_club_member_false,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,...,p_of_10_road_age_category_V40,p_of_10_road_age_category_V45,p_of_10_road_age_category_V50,p_of_10_road_age_category_V55,p_of_10_road_age_category_V65,p_of_10_road_age_category_V70,p_of_10_road_age_category_V75,p_of_10_road_age_category_V80,p_of_10_road_age_category_V85,p_of_10_road_age_category_V95
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [91]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create imputers for different columns
# low_value_imputer = SimpleImputer(strategy="constant", fill_value=np.inf)
seconds_value_imputer = SimpleImputer(strategy="constant", fill_value=400000)
low_value_imputer = SimpleImputer(strategy="constant", fill_value=400)
high_value_imputer = SimpleImputer(strategy="constant", fill_value=0) #-np.inf is too big for a float 
binary_value_imputer = SimpleImputer(strategy="constant")


# Combine the imputers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("low_value_good", low_value_imputer, low_value_good_columns),
        ("high_value_good", high_value_imputer, high_value_good_columns),
        ("binary", binary_value_imputer, binary_columns),
        
        # ("remaining", SimpleImputer(strategy="constant", fill_value=-1), "passthrough")
    ]
)

PIPELINE_STEP_PREPROCESSOR = "preprocessor"
PIPELINE_STEP_SCALER = "scaler"

# Create a pipeline with the preprocessor, scaler, and the logistic regression classifier
pipeline = Pipeline(steps=[
    (PIPELINE_STEP_PREPROCESSOR, preprocessor),
    (PIPELINE_STEP_SCALER, StandardScaler()),
    ("classifier", LogisticRegression(penalty=None))
])

# Fit the pipeline on the training data
pipeline.fit(pb_achieved_feature_train_df_pandas, pb_achieved_target_train_df_pandas["is_pb"])

# 
# # Transform the test data
# pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('low_value_good',
                                                  SimpleImputer(fill_value=400,
                                                                strategy='constant'),
                                                  ['best_position_year_to_date',
                                                   'average_position_year_to_date',
                                                   'worst_position_year_to_date',
                                                   'position_record_best_position_all_time',
                                                   'position_record_worst_position_all_time',
                                                   'prior_quickest_run_position',
                                                   'position_rec...
                                                   'p_of_10_road_age_category_TBC',
                                                   'p_of_10_road_age_category_U11',
                                                   'p_of_10_road_age_category_U13',
                                                   'p_of_10_road_age_category_U15',
                                                   'p_of_10_road_age_category_U17',
                                                   'p_of_10_road_age_category_U20',
                                                   'p_of_10_road_age_category_U23',
                                                   'p_of_10_road_age_category_V35',
                                                   'p_of_10_road_age_category_V40', ...])])),
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(penalty=None))])

## Run Pipeline Stages explicitly

To run the full pipeline 
`pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)`

However this does not work 
> AttributeError: This 'Pipeline' has no attribute 'transform'

this is because `LogisticRegression` does not have a `transform` method, but a `fit`. 
  


In [27]:
# Transform the test data using the preprocessor part of the pipeline
pb_achieved_feature_train_preprocessed_df = pipeline.named_steps['preprocessor'].transform(pb_achieved_feature_train_df_pandas)

# Scale the preprocessed test data
pb_achieved_feature_test_scaled_df = pipeline.named_steps['scaler'].transform(pb_achieved_feature_train_preprocessed_df)

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [28]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_test_scaled_df,
    pb_achieved_target_train_df_pandas["is_pb"],
)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [113]:


logistic_regression_classifier.classes_



array([False,  True])

In [114]:
import altair as alt

# Convert the set to a list to maintain order
all_feature_columns_list = list(all_feature_columns)

# Create a DataFrame with the features and their coefficients
feature_coefficients_df = pl.DataFrame(
    {
        "feature": all_feature_columns_list,
        "coefficient": logistic_regression_classifier.coef_[0][:len(all_feature_columns_list)],
    }
)

# Sort the DataFrame by coefficient in descending order
sorted_feature_coefficients_df = feature_coefficients_df.sort("coefficient", descending=True)

# Plot the bar chart
alt.Chart(sorted_feature_coefficients_df.to_pandas()).mark_bar().encode(
    x="coefficient",
    y=alt.Y("feature", sort="-x")
).properties(
    width=500,
    height=300,
).display()

alt.Chart(...)

In [87]:
logistic_regression_classifier.coef_

array([[ 0.02248893, -0.1604215 , -0.35113392,  0.19814715,  0.29265715,
         0.00387152,  0.02259435, -0.01800394,  0.19599449,  0.10406957,
        -0.13826141,  0.30100984, -0.80793519, -0.17211552,  0.19269861,
        -0.00908506, -0.03693306,  0.01017686,  0.02899874,  0.08223687,
         0.01387903,  0.04489769,  0.01410925, -0.07564159, -0.02787106,
         0.02740834,  0.06258948, -0.06138874, -0.049353  , -0.01010949,
         0.21542324, -0.55818381, -0.11816411,  0.04142924]])

In [88]:

# Make predictions on the test data
predictions = pipeline.named_steps['classifier'].predict(pb_achieved_feature_test_scaled_df)

# Print the predictions
print(predictions)

[False False  True ... False False False]


In [89]:
print(len(pb_achieved_feature_train_df_pandas))
print(len(pb_achieved_target_train_df_pandas))
print(len(pb_achieved_feature_test_df_pandas))
print(len(pb_achieved_target_test_df_pandas))

15710
15710
6733
6733


In [90]:
print(len(pb_achieved_feature_train_df_pandas.columns))
print(len(pb_achieved_target_train_df_pandas.columns))
print(len(pb_achieved_feature_test_df_pandas.columns))
print(len(pb_achieved_target_test_df_pandas.columns))

47
1
47
1
